In [26]:
import pandas as pd
import requests
import time
from pathlib import Path

In [27]:
# === Config ===
MODEL = "model-1"
EVAL_FILE = "../../data/processed/bdm-corpus-2/stage-3/test.csv"
TRAIN_FILE = "../../data/processed/bdm-corpus-2/stage-3/train.csv" # grab 6 examples for few shot
RESULT_FILE = "../../results/exp003/results.csv"
TEMPERATURE = 0.0

In [ ]:
eval_df = pd.read_csv(EVAL_FILE)
shot_df = pd.read_csv(TRAIN_FILE)

In [ ]:
# Sample few-shot examples (6)
fewshot_up = shot_df[shot_df["Direção"] == "Aumento"].sample(n=3, random_state=1) # prompts are reproducible with seed 1
fewshot_down = shot_df[shot_df["Direção"] == "Diminuição"].sample(n=3, random_state=1)
fewshot_df = pd.concat([fewshot_up, fewshot_down])

display(fewshot_df)

# === Step 2: Sample few-shot examples (6) ===
fewshot_up = eval_df[eval_df["Direção"] == "Aumento"].sample(n=3, random_state=1)
fewshot_down = eval_df[eval_df["Direção"] == "Diminuição"].sample(n=3, random_state=1)
fewshot_df = pd.concat([fewshot_up, fewshot_down])
fewshot_indices = fewshot_df.index

,DataHora,Manchete,Direção
2553,2024-12-19 14:32:00,Itaú BBA reforça recomendação de compra para I...,Aumento
1357,2024-12-06 17:00:00,MERCADOS: Petróleo recua após decisão da Opep ...,Aumento
1221,2024-12-05 16:11:00,Nomeação de novo primeiro-ministro ocorrerá no...,Aumento
383,2024-11-27 15:22:00,Isenção do IR até R$ 5 mil teria efeito fiscal...,Diminuição
744,2024-12-02 11:17:00,"Emendas de relator, por exemplo, podem ser ret...",Diminuição
179,2024-11-25 16:51:00,"WTI para janeiro recua 3,23%, a US$ 68,94 por ...",Diminuição


In [31]:
# Format few-shot block
def format_example(row):
    return f"""Manchete: {row['Manchete']}
Resposta: {row['Direção']}"""

FEWSHOT_BLOCK = "\n\n".join(format_example(row) for _, row in fewshot_df.iterrows())

In [32]:
# Build prompt for each headline
def build_fewshot_prompt(headline):
    return f"""{FEWSHOT_BLOCK}

    Preveja o impacto de curto prazo no USD/BRL:

    Manchete: {headline}

    Responda apenas com uma palavra: Aumento ou Diminuição."""

In [33]:
# Query Ollama
def query_ollama(model, prompt):
    res = requests.post("http://localhost:11434/api/chat", json={
        "model": model,
        "temperature": TEMPERATURE,
        "stream": False,
        "messages": [{"role": "user", "content": prompt}]
    })
    res.raise_for_status()
    return res.json()["message"]["content"].strip()

In [34]:
# Run inference
predictions = []
for i, row in eval_df.iterrows():
    prompt = build_fewshot_prompt(row["Manchete"])
    try:
        out = query_ollama(MODEL, prompt)
    except Exception:
        out = "Erro"
    predictions.append(out)
    time.sleep(0.25)

In [35]:
eval_df["Previsão"] = predictions
eval_df.to_csv(RESULT_FILE, index=False)
print(f"✅ Saved predictions to {RESULT_FILE}")

✅ Saved predictions to ../../results/exp003/results.csv


In [36]:
from sklearn.metrics import classification_report

# Reload the results file
preds = pd.read_csv("../../results/exp003/results.csv")

# Normalize predictions (just in case)
preds["Previsão"] = preds["Previsão"].str.strip().str.capitalize()
preds["Direção"] = preds["Direção"].str.strip().str.capitalize()

# Report
report = classification_report(
    preds["Direção"], preds["Previsão"],
    labels=["Aumento", "Diminuição"],
    target_names=["Aumento", "Diminuição"],
    digits=3
)

print("\nClassification Report:\n")
print(report)


Classification Report:

              precision    recall  f1-score   support

     Aumento      0.508     0.910     0.652       200
  Diminuição      0.571     0.120     0.198       200

    accuracy                          0.515       400
   macro avg      0.540     0.515     0.425       400
weighted avg      0.540     0.515     0.425       400



In [37]:
from sklearn.metrics import confusion_matrix
'''
[[TP FN]
 [FP TN]]
'''
cm = confusion_matrix(df["Direção"], df["Previsão"], labels=["Aumento", "Diminuição"])
print(cm)

[[182  18]
 [176  24]]
